## Xarray engine: writing back to GRIB

<div class="alert alert-warning">
<b>Warning:</b> converting back Xarray to GRIB is an experimental feature and is not yet fully supported. </div>

First, we get some example GRIB data and convert it into Xarray.

In [1]:
import earthkit.data as ekd

ds_fl = ekd.from_source("sample", "pl.grib")
ds_xr = ds_fl.to_xarray()
ds_xr

pl.grib:   0%|          | 0.00/48.8k [00:00<?, ?B/s]

From version 0.11.0 the default engine for to_xarray is 'earthkit'. Use engine=`cfgrib` to invoke the cfgrib engine.


<xarray.Dataset> Size: 176kB
Dimensions:                  (forecast_reference_time: 4, step: 2, levelist: 2,
                              latitude: 19, longitude: 36)
Coordinates:
  * forecast_reference_time  (forecast_reference_time) datetime64[ns] 32B 202...
  * step                     (step) timedelta64[ns] 16B 00:00:00 06:00:00
  * levelist                 (levelist) int64 16B 500 700
  * latitude                 (latitude) float64 152B 90.0 80.0 ... -80.0 -90.0
  * longitude                (longitude) float64 288B 0.0 10.0 ... 340.0 350.0
Data variables:
    r                        (forecast_reference_time, step, levelist, latitude, longitude) float64 88kB ...
    t                        (forecast_reference_time, step, levelist, latitude, longitude) float64 88kB ...
Attributes:
    class:        od
    stream:       oper
    levtype:      pl
    type:         fc
    expver:       0001
    date:         20240603
    time:         0
    domain:       g
    number:       0
    Conventions:  CF-1.8
    institution:  ECMWF

In [2]:
import xarray as xr 
xr.set_options(keep_attrs=True)

# modify values
ds_xr += 1

#### Using to_fieldlist()

First, we convert a single variable back into a GRIB fieldlist.

In [3]:
ds_fl1 = ds_xr["t"].earthkit.to_fieldlist()
ds_fl1.head()

,centre,shortName,typeOfLevel,level,dataDate,dataTime,stepRange,dataType,number,gridType
0,ecmf,t,isobaricInhPa,500,20240603,0,0,fc,0,regular_ll
1,ecmf,t,isobaricInhPa,700,20240603,0,0,fc,0,regular_ll
2,ecmf,t,isobaricInhPa,500,20240603,0,6,fc,0,regular_ll
3,ecmf,t,isobaricInhPa,700,20240603,0,6,fc,0,regular_ll
4,ecmf,t,isobaricInhPa,500,20240603,1200,0,fc,0,regular_ll


We check if the computation results were correctly written to the generated GRIB data.

In [4]:
# original GRIB data
print(ds_fl.sel(param="t", step=0, level=500)[0].values[0])
# GRIB data converted from the modified xarray object
print(ds_fl1.sel(param="t", step=0, level=500)[0].values[0])

250.22500610351562
251.22500610351562


Next, we convert back the whole dataset into a GRIB fieldlist.

In [5]:
ds_fl1 = ds_xr.earthkit.to_fieldlist()
ds_fl1.head()

,centre,shortName,typeOfLevel,level,dataDate,dataTime,stepRange,dataType,number,gridType
0,ecmf,r,isobaricInhPa,500,20240603,0,0,fc,0,regular_ll
1,ecmf,r,isobaricInhPa,700,20240603,0,0,fc,0,regular_ll
2,ecmf,r,isobaricInhPa,500,20240603,0,6,fc,0,regular_ll
3,ecmf,r,isobaricInhPa,700,20240603,0,6,fc,0,regular_ll
4,ecmf,r,isobaricInhPa,500,20240603,1200,0,fc,0,regular_ll


In [6]:
out_name = "_from_xr_1.grib"
ds_fl1.to_target("file", out_name)
# read back and check the save GRIB
ds_tmp = ekd.from_source("file", out_name)
ds_tmp[0]

GribField(r,500,20240603,0,0,0)

#### Using to_grib()

In [7]:
out_name = "_from_xr_2.grib"
ds_xr.earthkit.to_grib(out_name)
ds_tmp = ekd.from_source("file", out_name)
ds_tmp[0]

GribField(r,500,20240603,0,0,0)

In [8]:
ds_grib = ekd.from_source("sample", "pl.grib")
ds_xr = ds_grib.to_xarray()

ds_xr += 1

ds_grib1 = ds_xr.earthkit.to_fieldlist()
ds_grib1.head()

pl.grib:   0%|          | 0.00/48.8k [00:00<?, ?B/s]

,centre,shortName,typeOfLevel,level,dataDate,dataTime,stepRange,dataType,number,gridType
0,ecmf,r,isobaricInhPa,500,20240603,0,0,fc,0,regular_ll
1,ecmf,r,isobaricInhPa,700,20240603,0,0,fc,0,regular_ll
2,ecmf,r,isobaricInhPa,500,20240603,0,6,fc,0,regular_ll
3,ecmf,r,isobaricInhPa,700,20240603,0,6,fc,0,regular_ll
4,ecmf,r,isobaricInhPa,500,20240603,1200,0,fc,0,regular_ll
